对于线性规划问题：
matlab解法
[x,fval] = linprog(c,A,b,Aeq,beq,lb,ub)

In [ ]:
from scipy import optimize
import numpy as np
#求解函数
res = optimize.linprog(c,A,b,Aeq,beq,lb,ub)
#目标最小值
print(res.fun)
#最优解
print(res.x)

eg:

max z = 2x1+3x2-5x3

s.t: x1+x2+x3 =7

​	2x1-5x2+x3>=10

​	x1+3x2+x3<=12

​	x1,x2,x3>=0

In [11]:
from scipy import optimize
import numpy as np

c = np.array([[-2,-3,5]])

A = np.array([[-2,5,-1],[1,3,1]])
b = np.array([[-10],[12]])
Aeq = np.array([[1,1,1]])
beq = np.array([[7]])

res = optimize.linprog(c,A,b,Aeq,beq)

print('最大值为：',-res.fun)
print('最优解为：',res.x)

最大值为： 14.571428565645084
最优解为： [6.42857143e+00 5.71428571e-01 2.35900788e-10]


另一种解法 ： pulp

此方法可以直接用list 而非用array

In [5]:
import pulp
#目标函数系数
f = [2,3,-5]
#约束条件
A = [[-2,5,-1],[1,3,1]]
b = [[-10],[12]]

Aeq = [[1,1,1]]
beq = [[7]]

#确定最大最小问题 Max与Min更改即可
m = pulp.LpProblem(sense=pulp.LpMaximize)
#定义三个变量放在列表中 这里的lowBound代表lb 自变量的下限为0
x = [pulp.LpVariable(f'x{i}',lowBound=0) for i in [1,2,3]]
#lpDot这个方法 用于将两个列表对应位置的元素相乘之后再相加
#相当于 x[0]y[0]+x[1]y[1]

#在上面定义了m的类型是最大还是最小 现在再加上一个式子的要求
m += pulp.lpDot(f,x)

# 加上不等式的约束
for i in range(len(A)):
    m += (pulp.lpDot(A[i],x) <= b[i])
    
#加上等式约束
for i in range(len(Aeq)):
    m += pulp.lpDot(Aeq[i],x) == beq[i]

#求解
m.solve()
#这个求解过程 相当于在不断给m增加约束条件 m就是一个代表着约束的变量

#输出结果
print('优化结果：',pulp.value(m.objective))
print('参数取值：',[pulp.value(var) for var in x])

优化结果： 14.57142851
参数取值： [6.4285714, 0.57142857, 0.0]


In [22]:
#对于运输问题 
import pulp 
import numpy as np
from pprint import pprint

def transportation_problem(costs,x_max,y_max):
    #costs对应的是从a到b的运费 x_max对应的是产量 y_max对应的是需求量
    row = len(costs)
    col = len(costs[0])
    #定义问题
    prob = pulp.LpProblem('Transpotation_problem',sense = pulp.LpMaximize) #对于括号内第一部分内容是取名 第二部分为确定大小关系
    #定义变量
    var = [[pulp.LpVariable(f'x{i}{j}',lowBound=0,cat=pulp.LpInteger) for j in range(col)]for i in range(row)]
    #一个方法 变成一维list
    flatten = lambda x :[y for l in x for y in flatten(l)] if type(x) is list else [x] #里面的flatten是numpy里的
    #增加约束
    prob += pulp.lpDot(flatten(var),costs.flatten())  #第一个flatten是自己写的 第二个flatten是numpy的
    for i in range(row):
        prob += (pulp.lpSum(var[i]) <= x_max[i])  #对应的每一行的和（也就是一种商品销往所有地方的和）小于该商品的产量
    for j in range(col):
        prob += (pulp.lpSum([var[i][j] for i in range(row)]) == y_max[j])
    #求解
    prob.solve()
    return {'objective':pulp.value(prob.objective),'var':[[pulp.value(var[i][j])for j in range(col)]for i in range(row)]}

1.司守奎《数学建模算法与应用》
p7 例1.5

In [24]:
import pulp
import numpy as np
#目标函数
costs = np.array([[6,2,6,7,4,2,5,9],
        [4,9,5,3,8,5,8,2],
        [5,2,1,9,7,4,3,3],
        [7,6,7,3,9,2,7,1],
        [2,3,9,5,7,2,6,5],
        [5,5,2,2,8,1,4,3]])
x_max = [60,55,51,43,41,52]
y_max = [35,37,22,32,41,32,43,38]

transportation_problem(costs,x_max,y_max)


{'objective': 2151.0,
 'var': [[11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 38.0],
  [0.0, 37.0, 0.0, 0.0, 0.0, 13.0, 5.0, 0.0],
  [0.0, 0.0, 0.0, 32.0, 0.0, 19.0, 0.0, 0.0],
  [24.0, 0.0, 0.0, 0.0, 0.0, 0.0, 19.0, 0.0],
  [0.0, 0.0, 22.0, 0.0, 0.0, 0.0, 19.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 41.0, 0.0, 0.0, 0.0]]}